## Get a filtration from Gudhi

In [16]:
import gudhi as gd
import tadasets
sphere = tadasets.dsphere(n=25, d=2, r=1, noise=0, seed=42)
alpha = gd.AlphaComplex(points=sphere)
st = alpha.create_simplex_tree()
filtration = list(st.get_filtration())

## Convert it to boundary map data

In [17]:
from src.filtrations import simplices_by_dimension, compute_boundary_matrices, filtration_hash_map

unique_filtration_values = sorted(list(set([f for (_, f) in filtration])))
simplices_by_dim, simplices_by_dim_only_filt = simplices_by_dimension(filtration)
boundary_matrices = compute_boundary_matrices(simplices_by_dim)
boundary_maps_index_dict = filtration_hash_map(filtration, simplices_by_dim_only_filt)
boundary_matrices

[0.0, 0.0011069328884322914, 0.004371725216098328, 0.0047241730857375715, 0.014563230817399289, 0.0167160105645449, 0.020421464162303817, 0.02546175064832349, 0.026645590437182107, 0.026800551472400735, 0.027562930776185896, 0.028963960263729722, 0.029060695194725567, 0.03112329831866171, 0.034292745500686825, 0.03612815611372616, 0.03966461670406822, 0.0400698582122064, 0.04441796707591594, 0.04884436573081008, 0.052766590512513395, 0.06342968271978949, 0.06361154972196233, 0.08535724083291552, 0.08694565715396549, 0.0869456580426308, 0.08778532827051116, 0.0890609396175458, 0.09115194518810049, 0.09726154965896285, 0.10433332470593559, 0.10539997067827916, 0.10698466105235303, 0.1090147327013514, 0.113321226926027, 0.13168593074010293, 0.13470690124391815, 0.14309603259303164, 0.14457544064394706, 0.15115100789362687, 0.1516271641567509, 0.1516427338878758, 0.17163697847394072, 0.17243611270858217, 0.19174794328725325, 0.1923996547957571, 0.1950190126995972, 0.20002383689400555, 0.20

{1: {'n_rows': 25,
  'n_cols': 100,
  'data': array([ 1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
         -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
         -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
         -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
         -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
         -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
         -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,

## Compute homology using persistent laplacians
For now, compute only first homology.
Lanczos library panics on diagonal matrices it seems.

In [18]:
import persistent_laplacians
result = persistent_laplacians.process_tda(
    boundary_matrices,
    boundary_maps_index_dict
)

## Barcodes 

In [26]:
from src.barcodes import barcodes
barcodes = barcodes(result, unique_filtration_values)
print(barcodes)
barcodes = [
    (q, (unique_filtration_values[i], unique_filtration_values[j]))
    for q in barcodes.keys()
    for (i, j) in barcodes[q].keys()
]
sorted(barcodes)

{1: {(65, 84): 1, (7, 10): 1, (50, 51): 1, (82, 83): 1, (65, 80): -1, (24, 25): 1, (52, 53): 1, (67, 69): 1, (28, 31): 1, (64, 70): 1, (55, 56): 1, (64, 84): -1, (45, 46): 1, (40, 41): 1, (61, 74): 1, (44, 84): 1, (49, 71): 1, (37, 38): 1, (36, 79): 1, (64, 80): 1, (9, 13): 1, (60, 80): 1, (57, 59): 1}, 2: {(113, 115): 1, (89, 117): 1, (107, 110): 1, (112, 113): 1}, 3: {}}


[(1, (0.02546175064832349, 0.027562930776185896)),
 (1, (0.026800551472400735, 0.03112329831866171)),
 (1, (0.08694565715396549, 0.0869456580426308)),
 (1, (0.09115194518810049, 0.10539997067827916)),
 (1, (0.13470690124391815, 0.3812157210058862)),
 (1, (0.14309603259303164, 0.14457544064394706)),
 (1, (0.1516271641567509, 0.1516427338878758)),
 (1, (0.19174794328725325, 0.4619707011897518)),
 (1, (0.1923996547957571, 0.1950190126995972)),
 (1, (0.20625453740338456, 0.3331329806915774)),
 (1, (0.21427641695380947, 0.21429864552772004)),
 (1, (0.22000470980115436, 0.22376414942279665)),
 (1, (0.24584711568813583, 0.2462713216645328)),
 (1, (0.2513978894679636, 0.2550177343741832)),
 (1, (0.25717912745174704, 0.4016365338505521)),
 (1, (0.29339981192864806, 0.34968255183627167)),
 (1, (0.31205880308081213, 0.3312652681257092)),
 (1, (0.31205880308081213, 0.4016365338505521)),
 (1, (0.31205880308081213, 0.4619707011897518)),
 (1, (0.31311440247332867, 0.4016365338505521)),
 (1, (0.313114

## Verify with Gudhi

In [25]:
gudhi_persistence = [(q, (i, j)) for (q, (i,j)) in st.persistence() if q > 0]
sorted(gudhi_persistence)

[(1, (0.02546175064832349, 0.027562930776185896)),
 (1, (0.026800551472400735, 0.03112329831866171)),
 (1, (0.08694565715396549, 0.0869456580426308)),
 (1, (0.09115194518810049, 0.10539997067827916)),
 (1, (0.13470690124391815, 0.3812157210058862)),
 (1, (0.14309603259303164, 0.14457544064394706)),
 (1, (0.1516271641567509, 0.1516427338878758)),
 (1, (0.19174794328725325, 0.4619707011897518)),
 (1, (0.1923996547957571, 0.1950190126995972)),
 (1, (0.20625453740338456, 0.3331329806915774)),
 (1, (0.21427641695380947, 0.21429864552772004)),
 (1, (0.22000470980115436, 0.22376414942279665)),
 (1, (0.24584711568813583, 0.2462713216645328)),
 (1, (0.2513978894679636, 0.2550177343741832)),
 (1, (0.25717912745174704, 0.4016365338505521)),
 (1, (0.29339981192864806, 0.34968255183627167)),
 (1, (0.31205880308081213, 0.3312652681257092)),
 (1, (0.3263574834499623, 0.3283299029320257)),
 (1, (0.4177413648892114, 0.4431463402790963)),
 (2, (0.7010479799872495, 0.9999999999999999)),
 (2, (0.999999999